In [2]:
# Import libraries
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [107]:
# !pip install pixiedust

In [65]:
import pixiedust

Pixiedust database opened successfully
Table VERSION_TRACKER created successfully
Table METRICS_TRACKER created successfully

Share anonymous install statistics? (opt-out instructions)

PixieDust will record metadata on its environment the next time the package is installed or updated. The data is anonymized and aggregated to help plan for future releases, and records only the following values:

{
   "data_sent": currentDate,
   "runtime": "python",
   "application_version": currentPixiedustVersion,
   "space_id": nonIdentifyingUniqueId,
   "config": {
       "repository_id": "https://github.com/ibm-watson-data-lab/pixiedust",
       "target_runtimes": ["Data Science Experience"],
       "event_id": "web",
       "event_organizer": "dev-journeys"
   }
}
You can opt out by calling pixiedust.optOut() in a new cell.


Pixiedust runtime updated. Please restart kernel
Table USER_PREFERENCES created successfully
Table service_connections created successfully


In [3]:
# %reload_ext autoreload
# %autoreload 2 # autoreload reloads modules automatically before entering the execution of code typed.
%matplotlib inline

In [7]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page.status_code # A status_code of 200 means that the page downloaded successfully. 

200

In [8]:
soup = BeautifulSoup(page.content, 'html.parser')

In [9]:
table  = soup.find('table', { "class" : "wikitable sortable"})

1. **We will work on the columns first**

In [72]:
header_rows  = table.find_all('th')
header = [column_name.get_text().rstrip() for column_name in header_rows] # use rstrip() to remove all the trailing newline
header

['Postcode', 'Borough', 'Neighborhood']

2. **We proceed then on our data**
    1. Ignore cells with a borough that is Not assigned.
    2. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [105]:
tr = table.findAll(['tr'])
data = []
for cell in tr:
    row    = cell.find_all('td')
    if len(row) == 0: # skip emtpy row
        continue
    else:
        if row[1].text == 'Not assigned': # Ignore cells with a borough that is "Not assigned". 
            continue
        else:
            if row[2].text.rstrip()  == 'Not assigned': # If a cell has a borough but that cell has a "Not assigned" neighborhood
                row[2] = row[1]                                   # then the neighborhood will be the same as the borough.
    df_row = [x.get_text().rstrip() for x in row]
    data.append(df_row)    

In [106]:
df =  pd.DataFrame(data=data,columns=header)
df.head(10)

,Postcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park,Queen's Park
6,M9A,Downtown Toronto,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


In [24]:
# header

In [ ]:
# data_rows  = contentTable.find_all('tr')
# data = []
# for row in data_rows:
#     cols=row.find_all('td')
#     cols=[x.text.strip() for x in cols]
#     data.append(cols)
# #     # cols

In [ ]:
# data[0:3]

In [ ]:
# data_rows  = contentTable.find_all('tr')
# data_rows
# # data = []
# for row in data_rows:
    
#     print(row.text.strip())

# # #     cols=row.find_all('td')
# # #     cols=[x.text.strip() for x in row]
# # #     data.append(cols)
# # #     # cols